In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

def ajustar_valores(valor):
    try:
        valor = float(valor)
        return max(0, min(20, valor))
    except:
        return 0

def procesar_hoja(df, nombre_hoja):
    if df.empty:
        return f"Error: La hoja '{nombre_hoja}' está completamente vacía."
    df.iloc[:, 1:] = df.iloc[:, 1:].applymap(ajustar_valores)
    return df

def guardar_en_excel(df_predicciones):
    file_path = "resultados_predicciones.xlsx"
    df_predicciones.to_excel(file_path, sheet_name="Predicciones", index=False)

modelo_guardado_path = 'my_model.h5'
model_existente = os.path.exists(modelo_guardado_path)

if model_existente:
    print("Cargando modelo existente...")
    model = keras.models.load_model(modelo_guardado_path)

nombre_archivo = "Modelo_Excel.xlsx"

try:
    hojas = pd.read_excel(nombre_archivo, sheet_name=None, engine='openpyxl')
    dfs = []

    for nombre_hoja, df in hojas.items():
        print(f"Procesando hoja: {nombre_hoja}")
        df = procesar_hoja(df, nombre_hoja)
        dfs.append(df)

    df_predicciones = dfs[-1][['Alumno']].copy()

    for curso in dfs[0].columns[1:]:
        print(f"Entrenando modelo para: {curso}")

        X = []
        y = []

        for idx, df in enumerate(dfs[:-1]):
            features = df.drop(columns=["Alumno"]).values.tolist()
            targets = dfs[idx+1][curso].values
            X.extend(features)
            y.extend(targets)

        X = np.array(X)
        y = np.array(y)

        if model_existente and model.input_shape[1] == X.shape[1]:
            pass  # El modelo existente es compatible con los nuevos datos, no es necesario reconfigurarlo
        else:
            print("Creando un nuevo modelo...")
            model = keras.Sequential([
                keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
                keras.layers.Dropout(0.3),
                keras.layers.Dense(64, activation='relu'),
                keras.layers.Dropout(0.2),
                keras.layers.Dense(32, activation='relu'),
                keras.layers.Dense(1)
            ])
            model.compile(optimizer='adam', loss='mse', metrics=['mae'])

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

        predicciones = model.predict(dfs[-1].drop(columns=["Alumno"]).values)
        predicciones = np.clip(predicciones, 0, 20)
        df_predicciones[curso + ' Predicted'] = predicciones

    for curso in df_predicciones.columns[1:]:
        df_predicciones[curso] = df_predicciones[curso].apply(lambda x: round(x, 2))

    model.save(modelo_guardado_path)

except Exception as e:
    print(f"Error al leer el archivo: {e}")

Cargando modelo existente...
Procesando hoja: Primer Grado
Procesando hoja: Segundo Grado
Entrenando modelo para: Matematica
Creando un nuevo modelo...
Epoch 1/100
1/1 [==============================] - 1s 676ms/step - loss: 120.6881 - mae: 10.9858 - val_loss: 174.5295 - val_mae: 13.2110
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 101.8525 - mae: 10.0922 - val_loss: 165.0879 - val_mae: 12.8487
Epoch 3/100
1/1 [==============================] - 0s 23ms/step - loss: 100.8617 - mae: 10.0430 - val_loss: 153.7915 - val_mae: 12.4013
Epoch 4/100
1/1 [==============================] - 0s 22ms/step - loss: 85.5019 - mae: 9.2467 - val_loss: 142.8228 - val_mae: 11.9508
Epoch 5/100
1/1 [==============================] - 0s 23ms/step - loss: 79.5372 - mae: 8.9184 - val_loss: 131.9674 - val_mae: 11.4877
Epoch 6/100
1/1 [==============================] - 0s 23ms/step - loss: 123.1950 - mae: 11.0993 - val_loss: 120.6152 - val_mae: 10.9825
Epoch 7/100
1/1 [==================